<a href="https://colab.research.google.com/github/youngjinjae/pythontest/blob/main/0727_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# 필요한 모듈 import
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler # 정규화
# 어제는 accuracy만 구했다.
from sklearn.metrics import classification_report

# tensrflow 구현 밑줄 이여도 된다.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

# raw data loading
df = pd.read_csv('/content/drive/MyDrive/kcc 3기 colab home/data/mnist/train.csv')

# 독립변수와 종속 변수 분리
x_data = df.drop('label',axis=1, inplace=False).values
t_data = df['label'].values.reshape(-1,1)

# 정규화 진행
scaler = MinMaxScaler()
# 최대 최소 알아야 사용가능
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

# train 과 test 데이터 분리
x_data_train_norm, x_data_test_norm, t_data_train, t_data_test = \
train_test_split(x_data_norm,
                t_data,
                test_size=0.3,
                random_state=0)

# kears model 생성
keras_model = Sequential()

# layer 를추가
keras_model.add(Flatten(input_shape=(784,)))

## hiddenlayer deep #relu로 변경 actvi 수학식
keras_model.add(Dense(256, activation='relu'))
keras_model.add(Dense(128, activation='relu'))


keras_model.add(Dense(10, activation='softmax')) # class 표현 분류의 갯수

# softmax는 다중 분류시 그냥 분류시는 아님

# model 설정
keras_model.compile(optimizer=Adam(learning_rate=1e-2),
                    loss='sparse_categorical_crossentropy',
                   metrics=['accuracy']
                   )# 평가 metrics
#### deep
# 중간지점 저장 하는거나 함수 적절하게 대체해줌
from tensorflow.keras.callbacks import ModelCheckpoint
my_callback = ModelCheckpoint(filepath='tmp_checkpoint.ckpt',
                              save_weight_only=True,
                              save_best_only=True,
                              mointor='val_loss',
                              verbose=1
                              )
#조기종료에 대해 deep
from tensorflow.keras.callbacks import EarlyStopping # 조기 종료 모듈
earlystopping_callback = EarlyStopping(monitor='val_loss',
                                       patience=5,
                                       verbose=1,
                                       restore_best_weights=True
                                       )




# 파일 위치 저장, 저장, 어느성능에 따라 최고 높은거만 저장 best=flase는 폭에따라
# 모니터 val_loss , 1은 한줄씩~ call 잡음


# model 학습 call 추가 deep
keras_model_result = keras_model.fit(x_data_train_norm,
                                    t_data_train,
                                    epochs=100,
                                     batch_size=100,
                                     verbose=1,
                                     callbacks=[my_callback,earlystopping_callback],
                                     validation_split=0.2
                                    ) # 몇개 짜를거야 val 0% 평가
#89.3% 정도 # callback 추가 조지종료 early 해봣자 값 똑같다


Epoch 1/100
228/236 [===========================>..] - ETA: 0s - loss: 0.3225 - accuracy: 0.8999
Epoch 1: val_loss improved from inf to 0.20568, saving model to tmp_checkpoint.ckpt
236/236 [==============================] - 4s 8ms/step - loss: 0.3177 - accuracy: 0.9013 - val_loss: 0.2057 - val_accuracy: 0.9384
Epoch 2/100
228/236 [===========================>..] - ETA: 0s - loss: 0.1532 - accuracy: 0.9540
Epoch 2: val_loss improved from 0.20568 to 0.18883, saving model to tmp_checkpoint.ckpt
236/236 [==============================] - 2s 6ms/step - loss: 0.1534 - accuracy: 0.9541 - val_loss: 0.1888 - val_accuracy: 0.9468
Epoch 3/100
225/236 [===========================>..] - ETA: 0s - loss: 0.1089 - accuracy: 0.9657
Epoch 3: val_loss improved from 0.18883 to 0.18243, saving model to tmp_checkpoint.ckpt
236/236 [==============================] - 2s 6ms/step - loss: 0.1092 - accuracy: 0.9656 - val_loss: 0.1824 - val_accuracy: 0.9500
Epoch 4/100
223/236 [===========================>..] - E

In [3]:
# 모델 평가
print(keras_model.evaluate(x_data_test_norm,t_data_test)) #89.6%

import tensorflow as tf

print(classification_report(t_data_test,
 tf.argmax(keras_model.predict(x_data_test_norm),axis=1).numpy()))

# 정답 변수, 우리의 예측, 열아니면 행으로 찾아서 predict는 2차원으로
# tensorflow 파일 쓰면 안되고 numpy

394/394 [==============================] - 1s 4ms/step - loss: 0.8131 - accuracy: 0.9632
[0.8131428956985474, 0.96317458152771]
394/394 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1242
           1       0.98      0.99      0.98      1429
           2       0.96      0.97      0.97      1276
           3       0.97      0.95      0.96      1298
           4       0.97      0.96      0.97      1236
           5       0.98      0.94      0.96      1119
           6       0.97      0.98      0.98      1243
           7       0.98      0.96      0.97      1334
           8       0.86      0.98      0.92      1204
           9       0.98      0.92      0.95      1219

    accuracy                           0.96     12600
   macro avg       0.96      0.96      0.96     12600
weighted avg       0.96      0.96      0.96     12600



In [6]:
# 우리 모델 저장

keras_model.save('/content/drive/MyDrive/kcc 3기 colab home/my_model.h5')

In [ ]:
# 저장된 우리 모델을 불러보아요.
from tensorflow.keras.models import load_model

new_model = load_model('/content/drive/MyDrive/kcc 3기 colab home/my_model.h5')



In [12]:
# 만약 modelcheckpoint로 저장된 weight가 존재한다면 어떻게 모델 load 하나요

keras_model.load_weights('./tmp_checkpoint.ckpt')

